In [16]:
import Pyro5.api as pyro
from Pyro5.errors import PyroError

from bokeh.io import curdoc, output_notebook
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, Select
from bokeh.layouts import layout, column
from bokeh.plotting import figure

import pandas as pd

from datetime import datetime
from math import radians

Loading BokehJS ...

In [17]:
# Create periodic function
def update():
    global latest_timestamp

    rem_source = pyro.Proxy("PYROMETA:DT:GH_block,greenhouse:" + select_block_id.value[0] + ",block:" + select_block_id.value[1])
    new_data = pd.DataFrame(rem_source.get_sensor_log(select_sensor.value, seconds=30), columns=["values", "timestamp"])
    new_data["values"] = new_data["values"].astype(float)
    new_data["timestamp"] = new_data["timestamp"].apply(lambda x: datetime.fromisoformat(x))
    new_data = new_data[new_data["timestamp"] > latest_timestamp]
    latest_timestamp = new_data["timestamp"].min()

    source.stream(new_data.to_dict(orient="list"), rollover=500)
    print(f"{datetime.now()} --- Updated")
    print(new_data)

def change_source(attrname, old, new):
    rem_source = pyro.Proxy("PYROMETA:DT:GH_block,greenhouse:" + select_block_id.value[0] + ",block:" + select_block_id.value[1])
    new_data = pd.DataFrame(rem_source.get_sensor_log(select_sensor.value, hours=1), columns=["values", "timestamp"])
    new_data["values"] = new_data["values"].astype(float)
    new_data["timestamp"] = new_data["timestamp"].apply(lambda x: datetime.fromisoformat(x))
    source.data = new_data.to_dict(orient="list")
    plot.title.text = f"Block: {select_block_id.value}  Sensor: {select_sensor.value}"

# Plot creation
plot = figure(x_axis_type="datetime", width=900, height=350)

# Selector
sensor_options = [("AIRHUMIDITY", "Air humidity"), ("LUX", "Lux"), ("TEMPERATURE", "Temperature")]

select_sensor = Select(title="Sensor", value="TEMPERATURE", options=sensor_options)
select_sensor.on_change("value", change_source)

block_id_options = [("A1", "A1"), ("A2", "A2"), ("B1", "B1"), ("B2", "B2")]

select_block_id = Select(title="Greenhouse block", value="A1", options=block_id_options)
select_block_id.on_change("value", change_source)

plot.title.text = f"Block: {select_block_id.value}  Sensor: {select_sensor.value}"

# Obtaining the initial data
remote_source = pyro.Proxy("PYROMETA:DT:GH_block,greenhouse:" + select_block_id.value[0] + ",block:" + select_block_id.value[1])

success = False
base_log_df = pd.DataFrame([], columns=["values", "timestamp"])
while not success:
    try:
        base_log_df = pd.DataFrame(remote_source.get_sensor_log("TEMPERATURE", hours=1), columns=["values", "timestamp"])
        base_log_df["values"] = base_log_df["values"].astype(float)
        base_log_df["timestamp"] = base_log_df["timestamp"].apply(lambda x: datetime.fromisoformat(x))
        success = True
    except PyroError:
        print("Failed to get the initial set of data! Retrying . . .")

# Defining the data source object
source = ColumnDataSource(base_log_df.to_dict(orient="list"))

# Defining the kind of plots
plot.circle(x="timestamp", y="values", color="firebrick", line_color="firebrick", source=source)
plot.line(x="timestamp", y="values", source=source)

latest_timestamp = base_log_df["timestamp"].min()

date_pattern = ["%Y-%m-%d\n%H:%M:%S"]

plot.xaxis.formatter = DatetimeTickFormatter(seconds=date_pattern,
                                             minsec=date_pattern,
                                             minutes=date_pattern,
                                             hourmin=date_pattern,
                                             hours=date_pattern,
                                             days=date_pattern,
                                             months=date_pattern,
                                             years=date_pattern
                                             )


plot.xaxis.major_label_orientation = radians(80)
plot.xaxis.axis_label = "Datetime"
plot.yaxis.axis_label = "Values"

# Config layout
lay = layout(column(plot, select_block_id, select_sensor))
curdoc().add_root(lay)
curdoc().add_periodic_callback(update, 30000)

,values
timestamp,
2022-02-28 00:00:00,30.165386
2022-02-28 00:10:00,29.854472
2022-02-28 00:20:00,30.067606
2022-02-28 00:30:00,29.853643
2022-02-28 00:40:00,29.867686
2022-02-28 00:50:00,29.624211
2022-02-28 01:00:00,30.005288
